
![Deep Learning Türkiye Uygulamalar](https://raw.githubusercontent.com/deeplearningturkiye/pratik-derin-ogrenme-uygulamalari/84354f5989730fc297ec5c6d247b1990632cc992/images/dltr_logo.png)

[Google Colab'da çalıştır!](https://colab.research.google.com/drive/1WSiC13KMCL0-WVp7K1u4jb45T5fu9NEW)

<h2>Hiyerarşik Devirli Sinir Ağları (Hierarchical Recurrent Neural Networks (HRNN))</h2>

---



_Deep Learning Türkiye topluluğu tarafından hazırlanmıştır. <br>
__Amaç:__ El yazısı rakamların tanınması.<br>
__Veriseti:__ MNIST (http://yann.lecun.com/exdb/mnist/)<br>
__Algoritma:__ Hiyerarşik Devirli Sinir Ağları (Hierarchical Recurrent Neural Networks (HRNN))_<br>

El yazısı rakamların tanınmasında Hiyerarşik Devirli Sinir Ağları (HRNN) kullanımının bir örneğidir.
HRNN'ler karmaşık bir dizilim üzerinde geçici hiyerarşilerin birçok katmanını kullanarak öğrenebilirler.
Genelde, HRNN'in ilk devirli katmanı cümleleri (kelime vektörü gibi) cümle vektörlerine dönüştürüyor.
İkinci devirli katmanı sonrasında bu vektörlerin dizilimini bir döküman vektörüne dönüştürüyor.
Bu döküman vektörü ile hem kelime seviyesindeki hem de cümle seviyesindeki içerik yapısının korunmuş oluyor.

#### Referanslar
- [A Hierarchical Neural Autoencoder for Paragraphs and Documents](https://arxiv.org/abs/1506.01057)<br>
  Paragraf ve dökümanları HRNN ile dönüştürüyor.
  Sonuçların gösterdiğine göre HRNN standart RNN'lerin performansını geçmiş
  ve yazı özetlemek yada soru-cevap üretimi gibi daha ileri seviye görevlerde kullanılabilir.
- [Hierarchical recurrent neural network for skeleton based action recognition](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7298714)<br>
  İskelet bazında hareket tahmini konusunda şimdiye kadarki en iyi sonuçlara ulaşmıştır.
  3 katmanlı çift yönlü (bidirectional) HRNN ve tamamen bağlı katmanların (fully connected layers) kombinasyonu kullanılmıştır.

Aşağıdaki MNIST örneğinde, birinci LSTM katmanı öncelikle herbir (28,1) boyutundaki piksel sütununu (128,) boyutunda sütun vektörüne dönüştürüyor.
İkinci LSTM sonrasında bu (28, 128) boyutundaki 28 sütun vektörünü resim vektörüne yani tüm resme dönüştürüyor.
Son yoğun (Dense) katman tahmin yapmak için eklendi.

5 epoch sonunda, eğitim verisetinde __98.58%__, validasyon verisetinde __98.64%__ doğruluk oranı elde ediliyor.

In [0]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM
from keras.utils.np_utils import to_categorical # Bu satiri to_categorical fonksiyonunu hatasız kullanabilmek için ekledim

Using TensorFlow backend.


In [0]:
print("Kullanılan keras versiyonu: ",keras.__version__)

Kullanılan keras versiyonu:  2.1.6


In [0]:
# Parametrelerin değerlerinin belirlenmesi.
batch_size = 32 # her bir iterasyonda "32" resim alınsın
num_classes = 10  # ayırt etmek istediğimiz "10" rakam
epochs = 5 # eğitim 5 epoch sürsün


# Modele yerleştirme boyutları (Embedding Dimensions).
row_hidden = 128
col_hidden = 128

# Veriseti yükleniyor ve eğitim/test diye ayrılıyor.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Veri HRNN için 4 boyutlu olarak yeniden şekillendiriliyor.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11501568/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Sınıf vektörleri ikili (binary) forma dönüştürülüyor.
# "to_catogorical" fonksiyonu ile one-hot-encoding yapıyoruz.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [0]:
row, col, pixel = x_train.shape[1:]

# 4 boyutlu giriş verileri.
x = Input(shape=(row, col, pixel))

print(x)

# Her satırın pikselleri TimeDistributed Wrapper kullanılar dönüştürülüyor (encoding).
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Dönüştürülmüş satırların sütunları da dönüştürülüyor.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Son olarak, tahmin yapılıyor.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)

Tensor("input_2:0", shape=(?, 28, 28, 1), dtype=float32)


In [0]:
# Model kuruluyor.
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Eğitim yapılıyor.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
21888/60000 [=========>....................] - ETA: 10:05 - loss: 0.8785 - acc: 0.6907

59968/60000 [============================>.] - ETA: 0s - loss: 0.4923 - acc: 0.8338

60000/60000 [==============================] - 995s 17ms/step - loss: 0.4921 - acc: 0.8339 - val_loss: 0.1309 - val_acc: 0.9597
Epoch 2/5
10336/60000 [====>.........................] - ETA: 13:05 - loss: 0.1447 - acc: 0.9546

59968/60000 [============================>.] - ETA: 0s - loss: 0.1210 - acc: 0.9635

60000/60000 [==============================] - 987s 16ms/step - loss: 0.1210 - acc: 0.9635 - val_loss: 0.0817 - val_acc: 0.9762
Epoch 3/5
10336/60000 [====>.........................] - ETA: 12:56 - loss: 0.0809 - acc: 0.9763

...

60000/60000 [==============================] - 997s 17ms/step - loss: 0.0823 - acc: 0.9759 - val_loss: 0.0655 - val_acc: 0.9800
Epoch 4/5
10336/60000 [====>.........................] - ETA: 13:08 - loss: 0.0596 - acc: 0.9816

59968/60000 [============================>.] - ETA: 0s - loss: 0.0636 - acc: 0.9807

60000/60000 [==============================] - 998s 17ms/step - loss: 0.0637 - acc: 0.9807 - val_loss: 0.0918 - val_acc: 0.9709
Epoch 5/5
10336/60000 [====>.........................] - ETA: 13:09 - loss: 0.0599 - acc: 0.9800

59968/60000 [============================>.] - ETA: 0s - loss: 0.0536 - acc: 0.9840

60000/60000 [==============================] - 997s 17ms/step - loss: 0.0536 - acc: 0.9840 - val_loss: 0.0536 - val_acc: 0.9836


In [0]:
# Sonuçların değerlendirilmesi.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.053633743908256294
Test accuracy: 0.9836
